In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [3]:
orgs=pd.read_csv(r"/home/omkar/Omkar Pawar/Data Science/JOB APPLICATION TASKS/Anomaly Detection Exercise/assets/candidate_eval_04/candidate_eval_04/data/orgs.csv")
devices=pd.read_csv(r"/home/omkar/Omkar Pawar/Data Science/JOB APPLICATION TASKS/Anomaly Detection Exercise/assets/candidate_eval_04/candidate_eval_04/data/devices.csv")
auth=pd.read_csv(r"/home/omkar/Omkar Pawar/Data Science/JOB APPLICATION TASKS/Anomaly Detection Exercise/assets/candidate_eval_04/candidate_eval_04/data/auth.csv")
users=pd.read_csv(r"/home/omkar/Omkar Pawar/Data Science/JOB APPLICATION TASKS/Anomaly Detection Exercise/assets/candidate_eval_04/candidate_eval_04/data/users.csv")

In [4]:
devices.isna().sum()

device_id     0
type         29
os           57
dtype: int64

In [5]:
devices.duplicated().value_counts()

False    500
Name: count, dtype: int64

In [6]:
devices=devices.dropna()
devices=devices.drop_duplicates()
devices.isnull().sum()

device_id    0
type         0
os           0
dtype: int64

In [7]:
auth.isna().sum()

time            0
id              0
device_id       0
user_id         0
result          0
method       2566
country      2497
dtype: int64

In [8]:
duplicate_count = auth.duplicated().sum()
print("Number of duplicate rows:", duplicate_count)


Number of duplicate rows: 0


In [9]:
auth['method'].fillna(auth['method'].mode()[0], inplace=True)
auth['country'].fillna(auth['country'].mode()[0], inplace=True)


In [10]:
mdf = auth.merge(users, on='user_id').merge(devices, on='device_id').merge(orgs, on='organization_id')

In [11]:
# # Convert date and time columns to datetime
# mdf['start_date_x'] = pd.to_datetime(mdf['start_date_x'])
# mdf['start_date_y'] = pd.to_datetime(mdf['start_date_y'])
# mdf['Time'] = pd.to_datetime(mdf['time']).dt.time

# # Convert dates to timestamp values
# mdf['start_date_x_timestamp'] = (mdf['start_date_x'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
# mdf['start_date_y_timestamp'] = (mdf['start_date_y'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

# # Drop the original date columns
# mdf.drop(columns=['start_date_x', 'start_date_y'], inplace=True)

# # Convert time to timestamp values
# mdf['Time'] = mdf['Time'].apply(lambda t: t.hour * 3600 + t.minute * 60 + t.second)


In [12]:
# Convert 'time' column to datetime
mdf['time'] = pd.to_datetime(mdf['time'])

# Extract date features
mdf['Year'] = mdf['time'].dt.year
mdf['Month'] = mdf['time'].dt.month
mdf['Day'] = mdf['time'].dt.day
mdf['Time'] = mdf['time'].dt.time  # Extracts the time component

# Drop the original 'time' column if needed
mdf.drop(columns=['time'], inplace=True)

In [13]:
# Apply one-hot encoding to 'method', 'country', and 'os' columns
mdf = pd.get_dummies(mdf, columns=['method', 'country', 'os', 'type_x', 'type_y', 'result'], drop_first=True)

In [14]:
from sklearn.preprocessing import LabelEncoder
# Initialize label encoders
label_encoders = {}

# Apply label encoding to 'id', 'device_id', 'user_id', and 'organization_id'
categorical_columns = ['id', 'device_id', 'user_id', 'organization_id']
for column in categorical_columns:
    le = LabelEncoder()
    mdf[column] = le.fit_transform(mdf[column])
    label_encoders[column] = le

In [15]:
mdf.head()

,id,device_id,user_id,organization_id,start_date_x,start_date_y,Year,Month,Day,Time,...,country_US,os_ios,os_linux,os_macos,os_windows,type_x_laptop,type_x_mobile,type_y_llc,type_y_nonprofit,result_success
0,3441,23,98,0,2022-10-27,2022-03-25,2022,1,1,00:00:00,...,True,True,False,False,False,False,True,True,False,True
1,6680,267,98,0,2022-10-27,2022-03-25,2022,1,5,20:00:00,...,True,False,False,False,False,False,True,True,False,True
2,32783,267,98,0,2022-10-27,2022-03-25,2022,8,3,09:00:00,...,True,False,False,False,False,False,True,True,False,True
3,3554,267,55,0,2022-11-26,2022-03-25,2023,12,1,21:00:00,...,False,False,False,False,False,False,True,True,False,False
4,34081,97,98,0,2022-10-27,2022-03-25,2022,1,8,15:00:00,...,True,False,False,True,False,True,False,True,False,True


In [16]:
mdf.drop(columns=['start_date_x', 'start_date_y', 'Time'], inplace= True)

In [17]:
from sklearn.ensemble import IsolationForest
# Create the model
isolation_forest = IsolationForest(contamination=0.05)  # Contamination is the expected proportion of anomalies

# Fit the model
isolation_forest.fit(mdf)
# Predict anomalies (outliers)
predictions = isolation_forest.predict(mdf)

# The predictions array will contain 1 for normal instances and -1 for anomalies


/home/omkar/.local/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [18]:
# Assuming label_encoders contains the label encoders for 'user_id' and other categorical columns

# Get the indices of anomalies (-1 predictions)
anomaly_indices = [i for i, pred in enumerate(predictions) if pred == -1]

# Get the user IDs associated with anomalies
anomalous_user_ids = mdf.loc[anomaly_indices, 'user_id']

# Inverse transform the label-encoded user IDs
original_user_ids = label_encoders['user_id'].inverse_transform(anomalous_user_ids)

print("Original User IDs with anomalies:")
print(original_user_ids)


Original User IDs with anomalies:
['h8bjzx051' 'h8bjzx051' '95w2cxsmw' ... '0xeakun4p' '0xeakun4p'
 '0xeakun4p']


In [19]:
len(original_user_ids)

2222